In [1]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import AutoTokenizer
import pandas as pd

In [2]:
from scibox import switch_to_scibox, switch_to_huggingface
switch_to_huggingface()

In [3]:
! pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
! pip install -U accelerate
! pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
# Шаг 1: Подготовка данных
#df = pd.read_excel('clean_dataset_3_credit_ratings_agencies_v2.0.xlsx')
df = pd.read_excel('clean_train.xlsx')

In [6]:
df

,Id,pr_txt,Категория,Уровень рейтинга,pr_txt_cleaned,sentences_cleaned
0,2,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,подтвердить кредитный рейтинг компания уровень...
1,4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,присвоить кредитный рейтинг уровень март рейти...
2,7,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,AA,AA+,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,повысить кредитный рейтинг уровень март рейтин...
3,8,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,A,A,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,понизить кредитный рейтинг уровень изменить пр...
4,9,«Эксперт РА» повысил кредитный рейтинг компани...,BB,BB+,«Эксперт РА» повысил кредитный рейтинг компани...,повысить кредитный рейтинг компания уровень из...
...,...,...,...,...,...,...
1155,880,"НКР присвоило группе ""РКС Девелопмент"" кредитн...",BBB,BBB-,"Обоснование рейтингового действия Факторы, опр...",обоснование рейтинговый действие фактор опреде...
1156,908,НКР присвоило заводу КриалЭнергоСтрой кредитны...,BBB,BBB-,"Обоснование рейтингового действия Факторы, опр...",обоснование рейтинговый действие фактор опреде...
1157,978,НКР присвоило группе «Самолет» кредитный рейти...,A,A-,"Обоснование рейтингового действия Факторы, опр...",обоснование рейтинговый действие фактор опреде...
1158,1016,НКР присвоило «ПИК-Корпорации» кредитный рейти...,A,A+,"Обоснование рейтингового действия Факторы, опр...",обоснование рейтинговый действие фактор опреде...


In [7]:
# Шаг 2: Токенизация текста
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [8]:
# Шаг 3: Выбор модели (rubert-tiny2)
from transformers import AutoModelForSequenceClassification#, BertConfig

model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=7)  # 7 классов уровня рейтинга

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Шаг 4: Fine-tuning модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [10]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [11]:
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(df["Категория"])
df['labels'] = labels

df = df[['pr_txt_cleaned', 'labels']]
df.to_csv('CRA_train_1200.csv', index=False)
full_dataset = load_dataset('csv', data_files='CRA_train_1200.csv')

df.labels.nunique()

Generating train split: 0 examples [00:00, ? examples/s]

7

In [13]:
full_dataset = full_dataset['train'].train_test_split(test_size=0.2, seed = 17)
full_dataset

DatasetDict({
    train: Dataset({
        features: ['pr_txt_cleaned', 'labels'],
        num_rows: 928
    })
    test: Dataset({
        features: ['pr_txt_cleaned', 'labels'],
        num_rows: 232
    })
})

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset # (dataset format for transformers lib) split train/test; feed to learning algorithm; fast fucntion apply - .map()
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding #  to dynamically pad the sentences to the longest length in a batch during collation
# import evaluate # loading metric F1
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer # TrainingArguments and Trainer are helpful instead of using verbose vanilla pytorh training workflow
import numpy as np
import torch

In [15]:
dataset_tokenized = full_dataset.map(lambda examples: tokenizer(examples["pr_txt_cleaned"], truncation = True, max_length=2048, padding='max_length'), batched=True)

dataset_tokenized.set_format(type='torch', device=device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length = 2048)

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [16]:
from datasets import load_metric
from sklearn.utils.class_weight import compute_class_weight
import torch

In [17]:
class MulticlassTrainer(Trainer):
    # inheriting from Trainer class to override vanilla cross-entropy loss with weighted cross-enthropy loss
    def __init__(self, weights, *args, **kwds):
        super().__init__(*args, **kwds)
        self.weights = weights
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        #loss = cross_entropy(logits.squeeze(), labels.squeeze(), torch.from_numpy(self.weights))
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        #loss = log_loss(labels.squeeze(), logits.squeeze(), sample_weight = self.weights)
        return (loss, outputs) if return_outputs else loss

In [18]:
f1_metric = load_metric("f1", trust_remote_code=True)
def compute_metrics(eval_pred):
    # using macro f1
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average="macro")

/tmp/ipykernel_14351/2817711593.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric = load_metric("f1", trust_remote_code=True)


In [19]:
training_args = TrainingArguments(
    #output_dir="./drive/MyDrive/ML/Models/female_clothing_class", # checkpoints are saved here
    output_dir = "./out",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs = 60,
    warmup_steps=500,  # Количество шагов разогрева
    weight_decay=0.01,
    logging_dir="./runs",  # Каталог для логов TensorBoard
    logging_steps=50,  # Логирование каждые 400 шагов
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps = 1500,
    save_total_limit = 10, # max number of model checkpoints to safe
    load_best_model_at_end=True, # automatically loads the best model (save_steps and eval_steps must be multiples oh each other)
    metric_for_best_model="f1",  # Метрика для выбора лучшей модели
    #report_to='tensorboard', # it's also possible to report to weights & biases or other
    greater_is_better=True  # Указывает, что большее значение F1 лучше
)

# weights for unbalanced classes
weights = compute_class_weight(class_weight="balanced", classes=np.unique(dataset_tokenized["train"]["labels"].cpu().numpy()), y=dataset_tokenized["train"]["labels"].cpu().numpy())
WEIGHTS = torch.from_numpy(weights).float().to(device)

trainer = MulticlassTrainer(
    WEIGHTS,
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #callbacks = [tb]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
training_args.device # check the current device

device(type='cuda', index=0)

In [21]:
def print_cuda_memory():
    print("Total CUDA memory:", torch.cuda.get_device_properties(0).total_memory/(2**30))
    print("Allocated CUDA memory:", torch.cuda.memory_allocated()/(2**30))
    print("Cached CUDA memory:", torch.cuda.memory_reserved()/(2**30))

print_cuda_memory()

Total CUDA memory: 79.169677734375
Allocated CUDA memory: 0.10903215408325195
Cached CUDA memory: 0.126953125


In [22]:
trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss,Validation Loss,F1
50,1.948400,1.946969,0.051673
100,1.941200,1.935951,0.074785
150,1.924800,1.915184,0.078000
200,1.910500,1.892432,0.096755
250,1.882400,1.866293,0.093800
300,1.859500,1.849899,0.079631
350,1.873400,1.832311,0.157778
400,1.870300,1.810745,0.142780
450,1.806600,1.793448,0.175713
500,1.786400,1.751880,0.213680


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

TrainOutput(global_step=6960, training_loss=0.36187212182390877, metrics={'train_runtime': 902.7102, 'train_samples_per_second': 61.681, 'train_steps_per_second': 7.71, 'total_flos': 1643454144184320.0, 'train_loss': 0.36187212182390877, 'epoch': 60.0})

In [23]:
#df_test = pd.read_excel('clean_test.xlsx')
df_test = pd.read_excel('CRA_validation_110_answers.xlsx')
#X_test = df_test['pr_txt_cleaned']  # Входные данные - описание компании
#y_test = df_test['Категория']

In [24]:
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(df_test["Категория"])
df_test['labels'] = labels

df_test = df_test[['pr_txt', 'labels']]
df_test.to_csv('CRA_test_110_answers.csv', index=False)
test_dataset = load_dataset('csv', data_files='CRA_test_110_answers.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
#full_dataset = full_dataset.filter(lambda example: example['Review Text'] is not None and example['Recommended IND'] is not None)

df_test.labels.nunique()

Generating train split: 0 examples [00:00, ? examples/s]

7

In [25]:
test_dataset_tokenized = test_dataset.map(lambda examples: tokenizer(examples["pr_txt"], truncation = True, max_length=2048, padding='max_length'), batched=True)
# automatically truncate long sentences to be no longer than DistilBERT’s maximum input length

test_dataset_tokenized.set_format(type='torch', device=device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length = 2048)

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

In [26]:
preds = trainer.predict(test_dataset_tokenized['train'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [27]:
preds

PredictionOutput(predictions=array([[-2.12807202e+00, -3.90787911e+00, -5.44260144e-01,
        -4.85247932e-02,  6.71468115e+00, -9.43976760e-01,
        -9.92004871e-01],
       [ 5.63018918e-01,  6.64142084e+00, -1.81935179e+00,
        -1.91029751e+00, -2.99364018e+00,  1.15416765e+00,
        -7.17695951e-01],
       [-1.87608635e+00, -4.21275616e+00, -1.17727768e+00,
        -9.75859523e-01,  6.92421532e+00,  5.39640546e-01,
        -1.30190873e+00],
       [ 6.38750553e+00, -1.65200698e+00, -3.09478998e+00,
        -9.89487886e-01,  1.52845159e-01, -5.23266017e-01,
        -1.39624262e+00],
       [ 4.16835690e+00, -3.29812169e+00, -2.53669095e+00,
        -3.67311716e-01,  2.70940900e+00, -6.74333096e-01,
        -1.74616790e+00],
       [ 1.49865985e+00, -4.06721687e+00, -3.61190009e+00,
        -1.37240851e+00,  3.90869379e+00,  2.75568247e+00,
        -1.22052324e+00],
       [ 8.65659595e-01,  7.44091320e+00, -7.00417042e-01,
        -1.88672578e+00, -3.80693579e+00,  3.356

In [39]:
from sklearn.metrics import f1_score

micro_f1 = f1_score(df_test['labels'],preds[0].argmax(axis = 1), average='micro')
macro_f1 = f1_score(df_test['labels'],preds[0].argmax(axis = 1), average='macro')

micro_f1, macro_f1

(0.8090909090909091, 0.7595851570602402)

In [156]:
# 7 классов
# pr_txt f1 = 0.79
# pr_txt_cleaned f1 = 0.927
# sentences_cleaned f1 = 0.852

# 17 классов
# pr_txt_cleaned f1 = 0.457

In [40]:
!pip install lime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

class_names = ['AAA','AA', 'A', 'BBB', 'BB', 'B', 'C']

def predictor(texts):
    #dataset_tokenized = tokenizer(texts, return_tensors="pt", padding=True)
    dataset_tokenized = Dataset.from_dict(dict(tokenizer(texts, return_tensors="pt", padding=True)))
    #model.to('cpu')
    #outputs = model(dataset_tokenized)
    outputs = trainer.predict(dataset_tokenized)
    tensor_logits = torch.tensor(outputs[0])
    probas = F.softmax(tensor_logits).detach().numpy()
    return probas

# AA
text = '«Эксперт РА» присвоил кредитный рейтинг АО «Просвещение» на уровне <rating>  Москва, 27 декабря 2022 г.  Рейтинговое агентство «Эксперт РА» присвоило  рейтинг кредитоспособности  нефинансовой компании   АО «Просвещение»   на уровне <rating>. Прогноз по рейтингу - стабильный. ") «Просвещение»  — ведущая российская компания, функционирующая в сегменте образования и  предоставляющая продукты и услуги в области печатного и цифрового контента,  повышения квалификации и обучения детей и взрослых, поставок оборудования и  прикладного программного обеспечения в рамках интеграционных проектов в сфере  образования. Компания входит в перечень системообразующих предприятий России и  активно вовлечена в национальные и федеральные образовательные проекты.  Агентство  положительно оценивает рыночные позиции компании, особенно на ключевом для нее  издательском рынке, генерирующим большую часть выручки и EBITDA. Компания занимает первое место  как на рынке учебно-методической литературы (УМЛ), так и по общему тиражу  непериодических изданий среди всех издательств России. «Просвещение» активно  развивает цифровые продукты, что может дополнительно усилить её рыночные  позиции на рынке образования за счёт синергетических эффектов с издательским  бизнесом. Доля цифровых продуктов в выручке пока незначительна, но при этом  сегмент растёт более быстрыми темпами по сравнению с другими бизнесами. На  фактор рыночных позиций, по мнению агентства, оказывает давление концентрация  выручки на сегменте издательской деятельности в сфере образования, на который  приходится более 70% выручки.  В  рамках издательского бизнеса «Просвещение» осуществляет полный цикл по созданию  учебно-методических пособий и их реализации. Компания управляет процессом разработки  учебных пособий, владеет авторскими правами на них, организует их производство  в партнёрских типографиях, обеспечивает поставку учебных изданий в рамках  государственных контрактов с образовательными организациями. Более 40% поставок  приходится на коммерческих клиентов. Бизнес основан на выстраивании  долгосрочных партнёрских отношений с поставщиками и потребителями, поэтому у  компании нет собственных производственных или складских помещений.   «Просвещение»  также занимается оснащением школ учебным оборудованием. Благодаря широкой  партнёрской сети компания может полностью оснащать школы и специализированные  классы необходимым оборудованием, поставлять оборудование в большом объёме и  гарантировать его качество, что выгодно отличает её от более мелких конкурентов.  Рынок оснащения и оборудования школ сильно сегментирован и состоит из большого  количества мелких игроков, что подразумевает более активную конкуренцию.  Барьеры  для входа в отрасль оценены на умеренном уровне, однако новым игрокам рынка  сложно конкурировать по масштабу и экспертизе с «Просвещением», требующим  больших инвестиций в создание продуктов и услуг для образования. Рынок  демонстрирует высокую устойчивость к внешним шокам. Поддержку оценке  устойчивости к внешним шокам'
# BB
#text = '«Эксперт РА» подтвердил кредитный рейтинг компании ООО «CЕЛЛ-Сервис» на уровне <rating>   Москва, 17 февраля 2023 г.  Рейтинговое агентство «Эксперт РА» подтвердило  рейтинг кредитоспособности  нефинансовой компании   ООО «СЕЛЛ-Сервис»   на уровне <rating>. Прогноз по рейтингу - стабильный.  ") ООО  «СЕЛЛ-Сервис» (далее – компания) специализируется преимущественно на реализации  компонентов, используемых в кондитерской промышленности, а также поставляет  ингредиенты для предприятий молочного производства, производства напитков и  компаний сегмента HoReCa. Компания является поставщиком какао-продуктов,  пищевой химии, ароматизаторов, агар-агаров и пектинов, заквасочных культур,  красителей и карамельного колера, загустителей, наполнителей, замороженного  пюре и одноразовой посуды и упаковки Huhtamaki.   Бизнес-профиль  компании оказывает давление на уровень рейтинга. Подверженность  компании внешним шокам характеризуется как высокая ввиду наличия сильной  зависимости от импортируемых товаров – около 80% всех поставок. Основная часть  импорта приходится на Индонезию (47% всего импорта за 9 месяцев 2022 года),  Китай (28%) и Вьетнам (7%). Отрасль подвержена логистическим рискам на фоне  задержек морских поставок, увеличения сроков разгрузки в портах,  продолжающегося роста стоимости фрахта и недостаточной пропускной способности  железной дороги в России, при этом в условиях геополитического шока 2022 года  компании преимущественно удалось сохранить договорные отношения, сроки и  условия поставок с ключевыми поставщиками и нарастить ассортиментную матрицу. Агентство  отмечает наличие у компании законтрактованных объемов поставок на 2023 год по  основным товарным позициям, что позволяет планировать сроки поступления  продукции для перепродажи. Система расчетов с поставщиками предусматривает  предоплату, в то время как около 70% выручки формируется за счет заказов с  отсрочкой платежа. Законтрактованные с поставщиками объемы импортных закупок  основываются на ретроспективных объемах отгрузок с коэффициентом роста, но при  этом не обеспечены договорными обязательствами с будущими покупателями. Рынок  характеризуется низкими материальными барьерами для входа.  Агентство  оценивает рыночные и конкурентные позиции компании на среднем уровне. Ввиду  небольших текущих масштабов бизнеса позиции на рынках сбыта оцениваются  умеренно-негативно, доля компании по какао-порошку и агар-агарам на рынке  России составляет примерно по 6%, по остальным ключевым товарам не превышает 3%.  Ассортиментный портфель характеризуется умеренно-высокой степенью  диверсификации. Так на крупнейшую номенклатурную позицию – какао-порошок – пришлось  40% от общих продаж, лимонной кислоты – 18%, доли других позиций не превышают 6%.'
#print(tokenizer(text, return_tensors='pt', padding=True))

explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(text, predictor, num_features = 2048, num_samples=1000, top_labels = 7)

exp.show_in_notebook(text=text)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/tmp/ipykernel_14351/2333811124.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probas = F.softmax(tensor_logits).detach().numpy()
